# 엑셀 파일 업데이트

### 퇴사 날짜를 지정하는 컬럼 설정

In [24]:
# pandas 라이브러리를 pd라는 별칭으로 임포트
import pandas as pd

# 사용할 엑셀 파일 경로 지정
file_path = "employee.xlsx"

# pandas의 read_excel()함수를 사용: 엑셀 파일을 읽어오기
excel_data = pd.read_excel(file_path, sheet_name=None)

# 읽어온 엑셀 데이터에서 'Sheet1' 시트를 선택하여 DataFrame 형태로 저장
sheet1_df = excel_data['Sheet1']

# 데이터프레임 형태의 sheet1_df에 '퇴사 날짜'라는 새로운 열을 추가
# : 초기값은 비워둠
# : Not a Time: 시간 데이터가 없음을 의미
sheet1_df['퇴사 날짜'] = pd.NaT

# 변경된 Sheet1 데이터를 다시 excel_data에 저장
# : 원래의 Sheet1 데이터가 업데이트
excel_data['Sheet1'] = sheet1_df

# 변경된 데이터를 파일로 저장
with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    # 모든 시트를 순회
    for sheet_name, df in excel_data.items():
        # 각 시트를 파일에 저장, 시트이름은 각 시트이름 그대로
        df.to_excel(writer, sheet_name=sheet_name, index=False)

### 생성된 퇴사 날짜 열에 데이터(퇴사 날짜)를 추가하는 경우
### : 추가된 해당 행의 소속을 파악하여 그 소속의 고용보험 가입자 현황 시트로 
### : 퇴사 날짜의 열에 값이 추가되도록 설정

In [22]:
# pandas 라이브러리를 pd라는 별칭으로 임포트
import pandas as pd

# 사용할 엑셀 파일 경로 지정
file_path = "employee.xlsx"

# pandas의 read_excel()함수를 사용: 엑셀 파일을 읽어오기
excel_data = pd.read_excel(file_path, sheet_name=None)

# 읽어온 엑셀 데이터에서 'Sheet1' 시트를 선택하여 DataFrame 형태로 저장
sheet1_df = excel_data['Sheet1']

# 퇴사 날짜 열을 datetime 타입으로 변환 (YYYY-MM-DD hh:mm:ss)
# : 해당 열에 입력되는 문자열을 날짜로 변환하여 해석
# : 해당 열에 데이터를 추가할 때는 문자열로 입력
sheet1_df['퇴사 날짜'] = pd.to_datetime(sheet1_df['퇴사 날짜'], errors='coerce')

#퇴사 날짜와 소속에 따른 퇴사 인원 집계
# '퇴사 날짜'가 존재하는 행만 선택 
#  > '소속'과 '퇴사 날짜'의 월별로 그룹화하여 퇴사 인원 계산

# notna(): 누락되지 않은 값을 찾아내는 함수
leaving_counts = sheet1_df[sheet1_df['퇴사 날짜'].notna()].groupby(['소속', sheet1_df['퇴사 날짜'].dt.month]).size()
    
# 각 소속의 '고용보험 가입자 현황' 시트 업데이트 & '총계' 계산
for sheet_name, df in excel_data.items():
    # 시트 이름에 '고용보험 가입자 현황'이 포함되어 있는 경우
    if "고용보험 가입자 현황" in sheet_name:
        employment_insurance_df = df
        # 시트 이름에서 소속 부분을 추출
        affiliation = sheet_name.split('_고용보험 가입자 현황')[0]
        
    # 퇴사 인원 업데이트
    for month in range(1, 13):
        # 해당 소속과 월에 해당하는 퇴사 인원 수를 찾음
        count = leaving_counts.get((affiliation, month))
        # 퇴사 인원 수가 있으면 해당 월의 퇴사 인원 열에 업데이트
        if count is not None:
            emp_df.loc[emp_df['Unnamed: 0'] == '퇴사인원', f'{month}월'] = count
            
        # 변경된 데이터를 excel_data에 다시 저장
        excel_data[sheet_name] = emp_df

# 변경된 데이터를 파일로 저장
with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    # 모든 시트를 순회
    for sheet_name, df in excel_data.items():
        # 각 시트를 파일에 저장, 시트이름은 각 시트이름 그대로
        df.to_excel(writer, sheet_name=sheet_name, index=False)

KeyError: '퇴사 날짜'

In [25]:
# 엑셀 파일 읽기
file_path = "employee.xlsx"
excel_data = pd.read_excel(file_path, sheet_name=None)
sheet1_df = excel_data['Sheet1']

# '퇴사 날짜' 열을 datetime 타입으로 변환
sheet1_df['퇴사 날짜'] = pd.to_datetime(sheet1_df['퇴사 날짜'], errors='coerce')

# 퇴사 날짜와 소속에 따른 퇴사 인원 집계
leaving_counts = sheet1_df[sheet1_df['퇴사 날짜'].notna()].groupby(['소속', sheet1_df['퇴사 날짜'].dt.month]).size()

# 각 소속의 '고용보험 가입자 현황' 시트 업데이트 및 '총계' 계산
for sheet_name, df in excel_data.items():
    if "고용보험 가입자 현황" in sheet_name:
        employment_insurance_df = df
        affiliation = sheet_name.split('_')[0]

        # 퇴사 인원 업데이트
        for month in range(1, 13):
            count = leaving_counts.get((affiliation, month))
            if count is not None:
                employment_insurance_df.loc[employment_insurance_df['Unnamed: 0'] == '퇴사인원', f'{month}월'] = count

        excel_data[sheet_name] = employment_insurance_df

# 변경된 데이터를 파일로 저장
with pd.ExcelWriter(file_path, engine='openpyxl', mode='w') as writer:
    for sheet_name, df in excel_data.items():
        df.to_excel(writer, sheet_name=sheet_name, index=False)